In [1]:
import onnx
import onnxruntime as ort
import numpy as np

In [2]:
print(ort.__version__)

1.15.1


In [3]:
modelfile = "../models/x1_network.onnx"

In [4]:
onnx_model = onnx.load(modelfile)
onnx.checker.check_model(onnx_model)

In [5]:
ort_sess = ort.InferenceSession(modelfile)

In [6]:
input_names = [x.name for x in ort_sess.get_inputs()]
input_shapes = [x.shape for x in ort_sess.get_inputs()]
output_names = [x.name for x in ort_sess.get_outputs()]
output_shapes = [x.shape for x in ort_sess.get_outputs()]

In [7]:
print("Input names: ", input_names, "with shapes: ", input_shapes)
print("Output names: ", output_names, "with shapes: ", output_shapes)

Input names:  ['jet_features', 'track_features'] with shapes:  [[1, 2], ['n_tracks', 21]]
Output names:  ['pu', 'pc', 'pb'] with shapes:  [[1], [1], [1]]


In [8]:
# numbers are taken from: 
# https://gitlab.cern.ch/atlas/athena/-/merge_requests/69721

jet_features = np.array([[11516.2, 1.0935]], dtype=np.float32)
print(jet_features.shape)

track_features = np.array([-0.113795, 0.173364, -0.230979, -0.233828, 0.000914455, 1.1331, 1.3538, 0.00291881, 0.00241196, 1.61412e-05, 4, 8, 1, 1, 0, 0, 0, 0, 0, 0, 0, -0.126863, 0.00371974, 0.17269, 0.135184, -0.000857426, 0.904411, -0.0226558, 0.00404071, 0.00238144, 1.47576e-05, 4, 7, 1, 1, 0, 0, 0, 0, 0, 0, 1], dtype=np.float32)
track_features = track_features.reshape(-1, 21)
print(track_features.shape)

(1, 2)
(2, 21)


In [9]:
pu, pc, pb = ort_sess.run(output_names, {input_names[0]: jet_features, input_names[1]: track_features})

In [10]:
print(f"pu: {pu}")
print(f"pc: {pc}")
print(f"pb: {pb}")

pu: [0.8282893]
pc: [0.10061738]
pb: [0.07109336]


In [18]:
jet_features = np.array([[85507.8, -3.05748]], dtype=np.float32)
track_features = np.array([]*21, dtype=np.float32)
track_features = track_features.reshape(-1, 21)

In [21]:
track_features

array([], shape=(0, 21), dtype=float32)

In [19]:
pu, pc, pb = ort_sess.run(output_names, {input_names[0]: jet_features, input_names[1]: track_features})

In [20]:
print(f"pu: {pu}")
print(f"pc: {pc}")
print(f"pb: {pb}")

pu: [0.6114677]
pc: [0.26959956]
pb: [0.11893278]
